In [3]:
import pandas as pd

In [9]:
# read gtf file
# Define columns for the GTF file
columns = ["seqname", "source", "feature", "start", "end", "score", "strand", "frame", "attribute"]

# Read GTF file into a DataFrame
df = pd.read_table("/data01/private/resources/GRCh38_hg38/hg38_gene.gtf", comment="#", header=None, names=columns, dtype={"seqname": str, "attribute": str})

# Extract gene information
df_genes = df[df["feature"] == "gene"]

# Extract gene_id and gene_name from the "attribute" column
df_genes["gene_id"] = df_genes["attribute"].str.extract(r'gene_id "([^"]+)"')
df_genes["gene_name"] = df_genes["attribute"].str.extract(r'gene_name "([^"]+)"')

# Select relevant columns
df_genes = df_genes[["gene_id", "gene_name"]]

In [11]:
#convert to dictionary
dict_genes = df_genes.set_index('gene_id').to_dict()['gene_name']

In [35]:
# read all .dexseq_counts files from the directory and merge them into one file
import glob
dexseq_result = pd.DataFrame()
list_of_dexseq_counts = glob.glob('/home/ls/rachelcw/projects/rare_disease/data/dexseq/*.dexseq_counts')
for file in list_of_dexseq_counts:
    sample_name = file.split('/')[-1].split('.')[0]
    print(sample_name)
    df = pd.read_table(file,sep='\t', header=None, names=['gene_id_exon',sample_name])
    if dexseq_result.empty:
        dexseq_result = df
    else:
        dexseq_result = pd.merge(dexseq_result, df, how='outer', on='gene_id_exon')

ss


sr
sh


In [36]:
#drop rows that start with _
# ss = ss[~ss['gene_id_exon'].str.startswith('_')]
dexseq_result['gene_id'] = dexseq_result['gene_id_exon'].str.split(':', expand=True)[0]

for row in dexseq_result.itertuples():
    if '+' in row.gene_id:
        list_gene = row.gene_id.split('+')
        dexseq_result.at[row.Index, 'gene_name'] = [dict_genes[gene] for gene in list_gene]
    elif '_' in row.gene_id:
        dexseq_result.at[row.Index, 'gene_name'] = pd.NA
    else:
        dexseq_result.at[row.Index, 'gene_name'] = dict_genes[row.gene_id]


In [37]:
dexseq_result.head()

,gene_id_exon,ss,sr,sh,gene_id,gene_name
0,ENSG00000000003:001,0,0,0,ENSG00000000003,TSPAN6
1,ENSG00000000003:002,21,28,12,ENSG00000000003,TSPAN6
2,ENSG00000000003:003,1,27,18,ENSG00000000003,TSPAN6
3,ENSG00000000003:004,0,0,0,ENSG00000000003,TSPAN6
4,ENSG00000000003:005,1,12,14,ENSG00000000003,TSPAN6


In [22]:
#concat dexseq counts file by gene_id_exon

# ss['gene_name'] = ss.apply(lambda row: dict_genes[row['gene_id']], axis=1)

,gene_id_exon,count,gene_id,gene_name
664950,_ambiguous,66884,_ambiguous,NaN
664951,_ambiguous_readpair_position,0,_ambiguous_readpair_position,NaN
